In [1]:
'''
20251127:
    load llama2的语言神经元&计算ppl
'''

'\n20251127:\n    load llama2的语言神经元&计算ppl\n'

In [3]:
!pip install datasets

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 4.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 17.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 20.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.9/193.9 kB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.2
    Uninstalling tqdm-4.66.2:
      Successfully uninstalled tqdm-4.66.2
  Attempting uninstall: requests
   

In [2]:
import transformers


from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os
# download checkpoint
from accelerate import load_checkpoint_and_dispatch
from tqdm import tqdm

os.environ["TOKENIZERS_PARALLELISM"] = "false"  # To prevent long warnings :)

#from accelerate import load_checkpoint_and_dispatch

from accelerate import init_empty_weights
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer

import util
import importlib

importlib.reload(util)      # 只能 reload 模块本身
from util import calc_ppl, get_test_data   # reload 后再重新 import 函数

import copy

In [3]:
def load_model(checkpoint):
    config = AutoConfig.from_pretrained(checkpoint,trust_remote_code=True)
    print('checkpoint:', checkpoint)
    
    
    if True:
    
        device_map='cuda'
        model= AutoModelForCausalLM.from_pretrained(checkpoint, trust_remote_code=True, torch_dtype= torch.bfloat16,device_map=device_map ) # for download model weight
    
    else:
        with init_empty_weights():
            model = AutoModelForCausalLM.from_config(config, trust_remote_code=True)
    
        model = load_checkpoint_and_dispatch(
            model, checkpoint, device_map="auto", dtype=torch.bfloat16#, no_split_module_classes=["GPTJBlock"]
        )
    
    tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)#AutoTokenizer.from_pretrained("/home/work/lyftri/projects/model_zoo/compass_sea_13b_s4_merge2HF_org_convert_TP_1_PP_2",  trust_remote_code=True)#AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)

    return model, tokenizer

In [4]:




def get_mask_neuron_model(model, activation_mask_path, need_lang):

    if activation_mask_path:
        activation_masks = torch.load(activation_mask_path)
    else:
        activation_masks = [None]
    
    final_output = []
    if is_llama:
        languages = ["en", "zh", "fr", "es", "vi", "id", "ja"]
    else:
        languages = ["en", "zh", "fr", "es", "vi", "id"]
    
    need_lang = [need_lang]#['en', 'zh', 'vi']
    
    for activation_mask, mask_lang in zip(activation_masks, languages):
    
        if mask_lang not in need_lang:continue
            
        print(f'get mask =====lang:{mask_lang}=====')
    
        
        if activation_mask:
            def factory(mask):
                def llama_forward_org(self, x):
                    gate_up, _ = self.gate_up_proj(x)  # b, l, 2i
                    i = gate_up.size(-1)
                    activation = F.silu(gate_up[:, :, : i // 2])
                    activation.index_fill_(2, mask, 0)
                    x = activation * gate_up[:, :, i // 2 :]
                    x, _ = self.down_proj(x)
                    return x
                def llama_forward(self, x):
                    gate_ = self.gate_proj(x)  # b, l, 2i
                    i = gate_.size(-1)
                    activation = F.silu(gate_)
                    activation.index_fill_(2, mask, 0)
                    x = activation * self.up_proj(x)
                    x = self.down_proj(x)
                    return x
    
                def bloom_forward(self, x: torch.Tensor):
                    x, _ = self.dense_h_to_4h(x)
                    x = self.gelu_impl(x)
                    x.index_fill_(2, mask, 0)
                    x, _ = self.dense_4h_to_h(x)
                    return x
    
                if is_llama:
                    return llama_forward
                else:
                    return bloom_forward
    
            for i, layer_mask in enumerate(activation_mask):
                if is_llama:
                    obj = model.model.layers[i].mlp
                else:
                    obj = model.model.layers[i].mlp
                obj.forward = MethodType(factory(layer_mask.to('cuda')), obj)
    return model

In [7]:
import argparse
from types import MethodType

import numpy as np
import torch
import torch.nn.functional as F




model_path ='/root/autodl-fs/model_zoo/meta-llama/Llama-2-7b-hf'
activation_mask_path='/root/autodl-fs/Language-Specific-Neurons/LLaMA-2-7B.neuron.pth'

is_llama = bool(model_path.find('llama') >= 0)
model, tokenizer = load_model(model_path)


checkpoint: /root/autodl-fs/model_zoo/meta-llama/Llama-2-7b-hf


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
# load test data
test_en = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/en_radom_1000_test.jsonl')
test_vi = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/vi_radom_1000_test.jsonl')
test_zh = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/zh_radom_1000_test.jsonl')
               
# calc ppl
model_list= [
    (model, 'org_model'),
    (None, 'mask_en'),
    (None, 'mask_vi'),
    (None, 'mask_zh'),
]

data_list = [
    (test_en, 'en_data'),
    (test_vi, 'vi_data'),
    (test_zh, 'zh_data')
    
]

for i_model, model_name in model_list:
    print('*'*20)
    print('*'*20)
    print('model name:', model_name)

    if model_name != 'org_model':
        i_model = get_mask_neuron_model(model, activation_mask_path, model_name.split('_')[-1])

    for i_data, data_name in data_list:
        print('='*20)
        print('data name:', data_name)
        ppl_sum, ppl_count = calc_ppl(i_model, tokenizer, i_data, max_len= 2048)
        print('ppl:',ppl_sum/ppl_count )





********************
********************
model name: org_model
data name: en_data


100%|██████████| 1000/1000 [00:53<00:00, 18.75it/s]


ppl: tensor(11.5736, device='cuda:0')
data name: vi_data


100%|██████████| 1000/1000 [02:21<00:00,  7.09it/s]


ppl: tensor(14.1997, device='cuda:0')
data name: zh_data


...

ppl: tensor(11.3950, device='cuda:0')
data name: vi_data


100%|██████████| 1000/1000 [02:20<00:00,  7.10it/s]


ppl: tensor(13.4718, device='cuda:0')
data name: zh_data


100%|██████████| 1000/1000 [01:39<00:00, 10.01it/s]

ppl: tensor(10.2067, device='cuda:0')


In [8]:
# load test data
test_en = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/en_radom_1000_test.jsonl')
test_vi = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/vi_radom_1000_test.jsonl')
test_zh = get_test_data('/root/autodl-fs/LRP_data/test_ppl_data/zh_radom_1000_test.jsonl')
               
# calc ppl
model_list= [
    (model, 'org_model'),
    #(None, 'mask_en'),
    #(None, 'mask_vi'),
    (None, 'mask_zh'),
]

data_list = [
    (test_en, 'en_data'),
    (test_vi, 'vi_data'),
    (test_zh, 'zh_data')
    
]

for i_model, model_name in model_list:
    print('*'*20)
    print('*'*20)
    print('model name:', model_name)

    if model_name != 'org_model':
        i_model = get_mask_neuron_model(model, activation_mask_path, model_name.split('_')[-1])

    for i_data, data_name in data_list:
        print('='*20)
        print('data name:', data_name)
        ppl_sum, ppl_count = calc_ppl(i_model, tokenizer, i_data, max_len= 2048)
        print('ppl:',ppl_sum/ppl_count )





********************
********************
model name: org_model
data name: en_data


100%|██████████| 1000/1000 [00:52<00:00, 18.93it/s]


ppl: tensor(11.5736, device='cuda:0')
data name: vi_data


100%|██████████| 1000/1000 [02:21<00:00,  7.09it/s]


ppl: tensor(14.1997, device='cuda:0')
data name: zh_data


...

ppl: tensor(11.3950, device='cuda:0')
data name: vi_data


100%|██████████| 1000/1000 [02:20<00:00,  7.10it/s]


ppl: tensor(13.4718, device='cuda:0')
data name: zh_data


100%|██████████| 1000/1000 [01:39<00:00, 10.01it/s]

ppl: tensor(10.2067, device='cuda:0')
